In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from attention import AttentionLayer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import os

def get_latest_checkpoint(directory):
    checkpoints = [file for file in os.listdir(directory) if file.startswith("checkpoint-") and file.endswith(".model.keras")]
    if not checkpoints:
        return None  # No checkpoint files found
    latest_checkpoint = max(checkpoints)
    return os.path.join(directory, latest_checkpoint)

checkpoint_directory = "./outputs/checkpoints/"
latest_checkpoint_path = get_latest_checkpoint(checkpoint_directory)
model_path = latest_checkpoint_path
tokenizer_path = "./outputs/tokenizer.json"
with open(tokenizer_path, 'r', encoding='utf-8') as f:
    loaded_tokenizer_json = f.read()
    tokenizer = tokenizer_from_json(loaded_tokenizer_json)
# Load saved model
model = keras.models.load_model(model_path)
# # Rename the layer
# layer_to_rename = model.get_layer('input_layer_1')
# layer_to_rename._name = 'input_layer_unique_name'

model.summary()

In [ ]:
#=======================================================================
max_text_len=256 #word
max_summary_len=128 #word
batch_size = 60
latent_dim = 300
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
#=======================================================================

In [ ]:
import pandas as pd
df = pd.read_csv("text_summary_prepared.csv")

In [ ]:
text = df['text'][0:100]
summary = df['summary'][0:100]

text_sequence = pad_sequences(tokenizer.texts_to_sequences(text), maxlen=max_text_len, padding='post')
summary_sequence = pad_sequences(tokenizer.texts_to_sequences(summary), maxlen=max_summary_len, padding='post')

In [ ]:
text_sequence.shape

In [ ]:
# encoder_inputs = model.layers[0]
encoder_inputs = model.inputs[0]
# decoder_inputs = model.layers[3]
decoder_inputs = model.inputs[1]
dec_emb_layer=model.layers[5]
encoder_outputs, state_h, state_c = model.layers[6].output
decoder_lstm = model.layers[7]
attn_layer = model.layers[8]
decoder_dense = model.layers[10]

In [ ]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,), name="input_asdasd")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_asaddfg")
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim), name="input_asd5522d")

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
reverse_target_word_index=tokenizer.index_word
reverse_source_word_index=tokenizer.index_word
target_word_index=tokenizer.word_index

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
for i in range(0,100):
    print("Review:",seq2text(text_sequence[i]))
    print("Original summary:",seq2summary(summary_sequence[i]))
    print("Predicted summary:",decode_sequence(text_sequence[i].reshape(1,max_text_len)))
    print("\n")